In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline  

In [2]:
n_input_width = 28
n_input_height = 28
n_input_channel = 1
training_epochs = 100
batch_size = 500
learning_rate = 0.01

n_conv1_patch_size = 3 
n_conv1_filter = 84

n_output  = 10 # e.g. MNIST total classes (0-9 digits)

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input_width * n_input_height])
y = tf.placeholder(tf.float32, [None, n_output])

wc1 = tf.Variable(tf.random_normal([n_conv1_patch_size, n_conv1_patch_size, n_input_channel, n_conv1_filter], stddev=0.1))
bc1 = tf.Variable(tf.random_normal([n_conv1_filter], stddev=0.1))

wf1 = tf.Variable(tf.random_normal([(n_input_width/2)*(n_input_height/2)*n_conv1_filter, n_output], stddev=0.1))
bf1 = tf.Variable(tf.random_normal([n_output], stddev=0.1))

# Reshape input
input_r = tf.reshape(x, shape=[-1, n_input_width, n_input_width, 1])
# Convolution
conv = tf.nn.conv2d(input_r, wc1, strides=[1, 1, 1, 1], padding='SAME')
# Add-bias
bias = tf.nn.bias_add(conv, bc1)
# Pass ReLu
relu = tf.nn.relu(bias)
# Max-pooling
pool  = tf.nn.max_pool(relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
# Vectorize
dense = tf.reshape(pool, [-1, wf1.get_shape().as_list()[0]])
# Fully-connected layer
out = tf.add(tf.matmul(dense, wf1), bf1)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(out, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(out, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.initialize_all_variables()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


TypeError: DataType float32 for attr 'T' not in list of allowed values: int32, int64

In [8]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    for epoch in range(training_epochs):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss and accuracy
        loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                          y: batch_y
                                                          })
        print "Epoch  " ,epoch  ,", batch Loss= " + \
              "{:.6f}".format(loss) + ", Training Accuracy= " + \
              "{:.5f}".format(acc)
    print "Optimization Finished!"

    # Calculate accuracy mnist test images
    print "Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images,
                                      y: mnist.test.labels
                                      })

Epoch   0 , batch Loss= 9.698008, Training Accuracy= 0.21400
Epoch   1 , batch Loss= 13.315502, Training Accuracy= 0.10800
Epoch   2 , batch Loss= 6.820310, Training Accuracy= 0.18200
Epoch   3 , batch Loss= 6.497667, Training Accuracy= 0.22800
Epoch   4 , batch Loss= 6.326646, Training Accuracy= 0.36400
Epoch   5 , batch Loss= 6.054041, Training Accuracy= 0.32400
Epoch   6 , batch Loss= 4.208635, Training Accuracy= 0.37600
Epoch   7 , batch Loss= 2.933131, Training Accuracy= 0.52800
Epoch   8 , batch Loss= 2.522697, Training Accuracy= 0.39200
Epoch   9 , batch Loss= 1.602883, Training Accuracy= 0.52600
Epoch   10 , batch Loss= 1.322742, Training Accuracy= 0.58200
Epoch   11 , batch Loss= 1.133940, Training Accuracy= 0.60200
Epoch   12 , batch Loss= 1.157976, Training Accuracy= 0.60400
Epoch   13 , batch Loss= 1.147177, Training Accuracy= 0.61400
Epoch   14 , batch Loss= 1.041107, Training Accuracy= 0.67000
Epoch   15 , batch Loss= 0.880427, Training Accuracy= 0.73000
Epoch   16 , batc

ResourceExhaustedError: OOM when allocating tensor with shape[10000,28,28,84]
	 [[Node: BiasAdd_2 = BiasAdd[T=DT_FLOAT, data_format="NHWC", _device="/job:localhost/replica:0/task:0/cpu:0"](Conv2D_2, Variable_9/read)]]
Caused by op u'BiasAdd_2', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7/dist-packages/zmq/eventloop/ioloop.py", line 162, in start
    super(ZMQIOLoop, self).start()
  File "/usr/local/lib/python2.7/dist-pa